# Text data feature extraction

Here are parameters of the program user can easily change and estimate their impact on the performance.

In [1]:
USE_MY_METHOD = True
USE_STOP_WORDS = True
USE_EMOTICONS = False
USE_NEGATION = True

# If set to false, number of occurances of words is calculated
USE_BOOLEAN_REPRESENTATION = True

NUMBER_OF_REVIEWS_TO_ANALYZE = 100000
NUMBER_OF_POPULAR_WORDS_TO_USE = 1000

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
import pandas as pd

In [3]:
products_raw = pd.read_csv("../valt_sa_data/amazon_baby.csv")
products = products_raw[['review', 'rating']][0:NUMBER_OF_REVIEWS_TO_ANALYZE]

Let us see how the data looks like:

In [4]:
products

,review,rating
0,"These flannel wipes are OK, but in my opinion ...",3
1,it came early and was not disappointed. i love...,5
2,Very soft and comfortable and warmer than it l...,5
3,This is a product well worth the purchase. I ...,5
4,All of my kids have cried non-stop when I trie...,5
5,"When the Binky Fairy came to our house, we did...",5
6,"Lovely book, it's bound tightly so you may not...",4
7,Perfect for new parents. We were able to keep ...,5
8,A friend of mine pinned this product on Pinter...,5
9,This has been an easy way for my nanny to reco...,4


Let us explore a specific example of a baby product.

In [5]:
products.iloc[9]

review    This has been an easy way for my nanny to reco...
rating                                                    4
Name: 9, dtype: object

Let us define an emoticons extraction function.

In [6]:
emoticons = [
    ':)', ':))', ':)))', ':(', ':((',
    ':(((', '=)', '=(', '=))', '=(('
]

def extract_emoticons(text):
    emoticons_in_text = []
    for emoticon in emoticons:
        i = text.find(emoticon)
        if i > -1:
            emoticons_in_text.append(emoticon)
    return emoticons_in_text

The helper functions below are also useful.

In [7]:
punctuation_to_remove = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'

def remove_punctuation(text):
    return text.translate(None, punctuation_to_remove)

pos_dict = {
    'NN': 'n', 'VB': 'v', 'VBD': 'v', 'VBG': 'v', 
    'VBN': 'v', 'VBP': 'v', 'VBZ': 'v',
    'JJ': 'a', 'JJR': 'a', 'JJS': 'a', 'JJT': 'a'
}

def get_pos_for_lemmatirzer(brown_post):
    if not brown_post in pos_dict:
        return 'n'
    else:
        return pos_dict[brown_post]

Now let us define a more sophisticated function for review analysis.

First the punctuation is removed.

Then every word is pos tagged to prepare for lemmatization.

After that lemmatization is performed to find the root form of each word.

All the stop words are removed if the corresponding program parameter is set.
Also if set, emoticons are extracted and processed.

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 
 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 
 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 
 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 
 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 
 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 
 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 
 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no',
 'nor', 'not', "n't", 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 
 'will', 'just', 'don', 'should', 'now']

def analyze_review(text):
    if USE_EMOTICONS:
        emoticons_features = extract_emoticons(text)
    else:
        emoticons_features = []

    text_without_punctuation = remove_punctuation(text)
    tokens = nltk.word_tokenize(text_without_punctuation)
    tagged_tokens = nltk.pos_tag(tokens)
    tokens_prepared_for_lemmatization = [(t[0], get_pos_for_lemmatirzer(t[1])) for t in tagged_tokens]
    
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    
    not_count = 0
    words_after_not_count = 0
    for tpl in tokens_prepared_for_lemmatization:
        current_word = lemmatizer.lemmatize(tpl[0], tpl[1]).lower()
        if words_after_not_count > 2:
            not_count = 0
            words_after_not_count = 0
        if current_word == 'not' or current_word == "n't":
                not_count += 1
        elif (not USE_STOP_WORDS) or (not current_word in stop_words):
            if USE_NEGATION and not_count % 2 == 1:
                current_word = 'NOT_' + current_word
                words_after_not_count += 1
            lemmas.append('F_' + current_word) # F - meaning feature
    
    review_words = lemmas + emoticons_features
    return review_words

Now, we will perform text analysis.
We will also find and print most common words and total number of words in the dictionary.

In [9]:
analyzed_reviews = products['review'].apply(str).apply(analyze_review)

review_words_list = [] # conaints duplicates, so that count of each word can be calculated
review_dictionary = set()

for w_l in analyzed_reviews:
    for word in w_l:
        review_words_list.append(word)
        review_dictionary.add(word)

from collections import Counter

review_counter = Counter(review_words_list)
most_common_words = map(lambda x: x[0], review_counter.most_common(NUMBER_OF_POPULAR_WORDS_TO_USE))
print most_common_words
print len(review_dictionary)

["F_'s", 'F_use', 'F_baby', 'F_one', 'F_get', 'F_love', 'F_would', 'F_great', 'F_like', 'F_buy', 'F_old', u'F_month', 'F_easy', 'F_seat', 'F_time', u'F_make', 'F_little', 'F_product', 'F_son', u'F_good', u'F_go', 'F_work', 'F_well', 'F_also', 'F_really', 'F_put', 'F_daughter', u'F_look', u'F_take', 'F_bottle', 'F_keep', 'F_stroller', 'F_much', 'F_first', 'F_toy', 'F_back', 'F_still', 'F_need', 'F_purchase', 'F_could', 'F_fit', u'F_bag', 'F_think', u'F_thing', u'F_come', u'F_try', 'F_diaper', 'F_year', 'F_even', 'F_recommend', 'F_child', 'F_small', u'F_find', 'F_2', 'F_two', 'F_car', 'F_hold', 'F_around', "F_'m", u'F_nice', "F_'ve", u'F_want', 'F_big', 'F_since', 'F_give', 'F_size', 'F_way', u'F_say', 'F_crib', 'F_pump', 'F_day', 'F_price', 'F_side', 'F_play', 'F_sleep', 'F_perfect', 'F_clean', u'F_seem', 'F_3', 'F_room', u'F_cup', 'F_new', 'F_cover', u'F_color', 'F_wash', u'F_set', u'F_soft', 'F_gate', 'F_chair', 'F_see', 'F_enough', u'F_review', u'F_lot', 'F_problem', 'F_every', 'F_ea

We perform feature extraction on the analyzed text. The matrix for machine learning is formed. Only features stored in the variable `significant_words` are included.

In [10]:
if USE_MY_METHOD:
    if USE_EMOTICONS:
        significant_words = most_common_words + emoticons
    else:
        significant_words = most_common_words
else:
    significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
                         'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
                         'work', 'product', 'money', 'would', 'return']
        
def count_number_of_significant_words(text):
    words = text['review']
    word_dict = {}
    for word in significant_words:
        word_dict[word] = 0
    for word in words:
        if word in significant_words:
            if not word in word_dict:
                word_dict[word] = 1
            else:
                if USE_BOOLEAN_REPRESENTATION:
                    word_dict[word] = 1
                else:
                    word_dict[word] = word_dict[word] + 1
    significant_words_counts = []
    for word in significant_words:
        significant_words_counts.append(word_dict[word]) 
    return pd.Series(significant_words_counts, index=significant_words)

word_counts_df = pd.DataFrame(analyzed_reviews).apply(count_number_of_significant_words, axis=1)
word_counts_df.columns = significant_words

products_with_words = products.join(word_counts_df)

Now, let us explore what the sample looks like after all the transformations.

The resulting matrix is very sparse, as was expected.

In [11]:
products_with_words.iloc[9]

review           This has been an easy way for my nanny to reco...
rating                                                           4
F_'s                                                             0
F_use                                                            0
F_baby                                                           1
F_one                                                            1
F_get                                                            0
F_love                                                           0
F_would                                                          0
F_great                                                          0
F_like                                                           0
F_buy                                                            0
F_old                                                            0
F_month                                                          0
F_easy                                                        

## Save prepared data into a file

In [12]:
X = products_with_words[significant_words]
y = products_with_words['rating']
X.to_csv('../valt_sa_data/x_m.csv', index=False)
y.to_csv('../valt_sa_data/y_m.csv', index=False)